<a href="https://colab.research.google.com/github/JuanDiaz77/Proyecto-colab/blob/main/Implementaci%C3%B3n_y_comparaci%C3%B3n_de_modelos_ensemble_con_datasets_p%C3%BAblicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# 1 Selección y carga del dataset
# =========================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar dataset Titanic desde GitHub
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Mostrar primeras filas
print(df.head())

# Tipos de variables
print(df.dtypes)

# Resumen estadístico
print(df.describe())

# =========================
# 2️ Exploración y preprocesamiento
# =========================

# Valores faltantes
print(df.isnull().sum())

# Manejo de valores faltantes
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df.drop(['Cabin', 'Ticket', 'Name', 'PassengerId'], axis=1, inplace=True)

# Variables categóricas -> One-Hot Encoding
df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)

# Separar variables predictoras y objetivo
X = df.drop('Survived', axis=1)
y = df['Survived']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# =========================
# 3️ Entrenamiento de modelos base
# =========================

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Gradient Boosting
gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

# XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# =========================
# 4️ Evaluación y comparación
# =========================

# Función para obtener métricas
def obtener_metricas(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1-score': f1_score(y_true, y_pred)
    }

# Obtener métricas
metricas_rf = obtener_metricas(y_test, y_pred_rf)
metricas_gb = obtener_metricas(y_test, y_pred_gb)
metricas_xgb = obtener_metricas(y_test, y_pred_xgb)

# Tabla comparativa
df_metricas = pd.DataFrame([metricas_rf, metricas_gb, metricas_xgb],
                            index=['Random Forest', 'Gradient Boosting', 'XGBoost'])
print(df_metricas)

# Reportes completos
print("Random Forest Report:\n", classification_report(y_test, y_pred_rf))
print("Gradient Boosting Report:\n", classification_report(y_test, y_pred_gb))
print("XGBoost Report:\n", classification_report(y_test, y_pred_xgb))

# =========================
# 5️ Ajuste básico de hiperparámetros (Random Forest ejemplo)
# =========================
rf_tuned = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42)
rf_tuned.fit(X_train, y_train)
y_pred_rf_tuned = rf_tuned.predict(X_test)

# Comparar métricas
metricas_rf_tuned = obtener_metricas(y_test, y_pred_rf_tuned)
print("Métricas Random Forest ajustado:", metricas_rf_tuned)

# Tabla comparativa antes/después
df_comparacion = pd.DataFrame([metricas_rf, metricas_rf_tuned], index=['RF Base', 'RF Ajustado'])
print(df_comparacion)

# =========================
# 6️ Análisis final (Markdown)
# =========================
# Puedes agregar celdas Markdown en el notebook con el siguiente contenido:

"""
# Análisis Final

- **Mejor desempeño:** El modelo XGBoost tuvo el mejor desempeño general según las métricas de Accuracy y F1-score, probablemente debido a su capacidad de manejar relaciones no lineales y regularización incorporada.
- **Ajuste de hiperparámetros:** Ajustar `n_estimators` y `max_depth` en Random Forest mejoró ligeramente las métricas, mostrando que el control de complejidad ayuda a reducir overfitting.
- **Dificultades encontradas:** Manejo de valores faltantes y codificación de variables categóricas fueron pasos clave. Elegir el método adecuado para imputar valores fue crítico.
- **Recomendaciones futuras:** Probar técnicas de ensemble más avanzadas como Stacking, ajustar más hiperparámetros usando GridSearchCV, y explorar feature engineering adicional para mejorar desempeño.
"""
